# Vector representations of 13,000 jobs

The O\*Net Database is the nation's largest occupational database. I used its information to compare jobs on skills required.

In this notebook, we will explore how the search function on O\*Net's website works. You can search for jobs at https://www.onetonline.org/find/. Once you search a job title, O\*Net returns the 20 closest matches to the position, ordered by decreasing similarity. Here is a link to the returned list for when one searches "Data Scientist": https://www.onetonline.org/find/result?s=Data+Scientist.

Before starting the notebook, we can answer a few questions about the search function by playing around with it.

1. Is there a common format for URL's when searching? 
A: Yes. For example, for Statisticians, the search link is https://www.onetonline.org/find/quick?s=Statisticians. In general, to search (job_title), the list is available at https://www.onetonline.org/find/quick?s=(job_title).

2. Are the jobs that pop up always in the list of 967 occupations described in the skills, abilities, and knowledge datasets?
A: Yes.

3. If you input a job included in the 967 companies, will it always be the first hit?
A: Yes.

4. How is the data collected?
It was originally created by occupation analysts. It is now populated with surveys completed by worker populations and occupation experts.

## Purpose of this notebook

O\*Net contains numerical ratings on the skills, abilities, and knowledge of 976 job titles. This naturally provides me with vector representations on all of these job titles. 

However, over 13,000 job titles are represented in my Glassdoor reviews dataset. How can I generalize from my 967 representations to 13,000?

The idea is that I can use the `requests` library to search each of my 13,000 job titles. Because of the positive answers to the first three questions above, I can represent each of my 13,000 by the average of some of the representations of the closest searched job titles.

In [7]:
#over 240 columns of skills, abilities, knowledge
import pandas as pd
pd.set_option('display.max_columns', 500) 

import numpy as np

%matplotlib inline 
import matplotlib as mpl
import matplotlib.pyplot as plt

from datetime import datetime
import time

from bs4 import BeautifulSoup
import requests

from inflection import singularize

from nose.tools import assert_equal

## Import "skills required" data for O\*Net's 967 job titles

In [4]:
#skills, abilities, knowledge ratings for 967 jobs
skills_abilities_knowledge = pd.read_csv('skills_abilities_knowledge_967_companies.csv', 
                                         index_col='Unnamed: 0')

#will not use survey sample counts, so drop columns
skills_abilities_knowledge = skills_abilities_knowledge.drop(['skills_N',
                                                              'abilities_N',
                                                              'knowledge N'], 
                                                             axis=1)

In [10]:
print('Shape of skills required DataFrame: {}'.format(skills_abilities_knowledge.shape))

skills_abilities_knowledge.head()

Shape of skills required DataFrame: (967, 241)


,Title,skill Reading Comprehension Importance,skill Reading Comprehension Level,skill Active Listening Importance,skill Active Listening Level,skill Writing Importance,skill Writing Level,skill Speaking Importance,skill Speaking Level,skill Mathematics Importance,skill Mathematics Level,skill Science Importance,skill Science Level,skill Critical Thinking Importance,skill Critical Thinking Level,skill Active Learning Importance,skill Active Learning Level,skill Learning Strategies Importance,skill Learning Strategies Level,skill Monitoring Importance,skill Monitoring Level,skill Social Perceptiveness Importance,skill Social Perceptiveness Level,skill Coordination Importance,skill Coordination Level,skill Persuasion Importance,skill Persuasion Level,skill Negotiation Importance,skill Negotiation Level,skill Instructing Importance,skill Instructing Level,skill Service Orientation Importance,skill Service Orientation Level,skill Complex Problem Solving Importance,skill Complex Problem Solving Level,skill Operations Analysis Importance,skill Operations Analysis Level,skill Technology Design Importance,skill Technology Design Level,skill Equipment Selection Importance,skill Equipment Selection Level,skill Installation Importance,skill Installation Level,skill Programming Importance,skill Programming Level,skill Operation Monitoring Importance,skill Operation Monitoring Level,skill Operation and Control Importance,skill Operation and Control Level,skill Equipment Maintenance Importance,skill Equipment Maintenance Level,skill Troubleshooting Importance,skill Troubleshooting Level,skill Repairing Importance,skill Repairing Level,skill Quality Control Analysis Importance,skill Quality Control Analysis Level,skill Judgment and Decision Making Importance,skill Judgment and Decision Making Level,skill Systems Analysis Importance,skill Systems Analysis Level,skill Systems Evaluation Importance,skill Systems Evaluation Level,skill Time Management Importance,skill Time Management Level,skill Management of Financial Resources Importance,skill Management of Financial Resources Level,skill Management of Material Resources Importance,skill Management of Material Resources Level,skill Management of Personnel Resources Importance,skill Management of Personnel Resources Level,abilities Oral Comprehension Importance,abilities Oral Comprehension Level,abilities Written Comprehension Importance,abilities Written Comprehension Level,abilities Oral Expression Importance,abilities Oral Expression Level,abilities Written Expression Importance,abilities Written Expression Level,abilities Fluency of Ideas Importance,abilities Fluency of Ideas Level,abilities Originality Importance,abilities Originality Level,abilities Problem Sensitivity Importance,abilities Problem Sensitivity Level,abilities Deductive Reasoning Importance,abilities Deductive Reasoning Level,abilities Inductive Reasoning Importance,abilities Inductive Reasoning Level,abilities Information Ordering Importance,abilities Information Ordering Level,abilities Category Flexibility Importance,abilities Category Flexibility Level,abilities Mathematical Reasoning Importance,abilities Mathematical Reasoning Level,abilities Number Facility Importance,abilities Number Facility Level,abilities Memorization Importance,abilities Memorization Level,abilities Speed of Closure Importance,abilities Speed of Closure Level,abilities Flexibility of Closure Importance,abilities Flexibility of Closure Level,abilities Perceptual Speed Importance,abilities Perceptual Speed Level,abilities Spatial Orientation Importance,abilities Spatial Orientation Level,abilities Visualization Importance,abilities Visualization Level,abilities Selective Attention Importance,abilities Selective Attention Level,abilities Time Sharing Importance,abilities Time Sharing Level,abilities Arm-Hand Steadiness Importance,abilities Arm-Hand Steadiness Level,abilities Manual Dexterity Importance,abilities Manual Dexterity Level,abilities Finge

In [6]:
#list of all 967 job titles
titles = skills_abilities_knowledge.loc[:,'Title'].tolist()

## Representing new jobs as vectors

I now write functions that will find the vector representations of job titles outside of our 967. I will test it for the title of 'Mathematician.'

In [11]:
def get_top_job_matches(title, num=3):
    '''
    Find the closest num job titles in database to an inputted job title.
    
    Args:
    title: job title to search
    num: number of jobs to return (at most 20)
    
    Returns:
    List of closest num job titles from database 
        (if exact match, return list with just that title)
    '''
    search_url = 'https://www.onetonline.org/find/quick?s=' + title
    html = requests.get(search_url).content #get html of search results page
    
    #find top (num) jobs suggested
    soup = BeautifulSoup(html,'lxml')
    divs = soup.find_all('td',{'class':'report2ed'})[:num]

    closest_jobs = [str(a) for div in divs for a in div.find('a')]

    #check if it is in Database by seeing if it matches first of closest_20_jobs (up to case, plural)
    if singularize(closest_jobs[0].lower()) == singularize(title.lower()): #title is in Database
        return [closest_jobs[0]]
    
    #otherwise return a number of jobs
    else:
        return closest_jobs

In [20]:
#find closest jobs to Mathematician
#also check what happens with misspellings
close_jobs_list = get_top_job_matches('Mathematicia')

#keys: rank as int
#value: vector representation of job title of that rank
close_jobs_vectors_dict = {}

for idx in range(len(close_jobs_list)):
    close_jobs_vectors_dict[idx] = \
        skills_abilities_knowledge[skills_abilities_knowledge['Title'] == close_jobs_list[idx]] \
        .drop('Title', axis=1)

In [21]:
def get_title_vector(title):
    '''
    Get representation for new job.
    
    Arg: 
    title: job title 
    
    Output:
    240 dimensional vector
    '''
    
    #closest 3 jobs to title, as vectors
    close_jobs = get_top_job_matches(title)
    
    
    #return the average of these vectors
    return pd.concat([skills_abilities_knowledge[skills_abilities_knowledge['Title'] == close_jobs[idx]] \
                      .drop('Title',axis=1)
                     for idx in range(len(close_jobs))]).mean(axis=0)

In [14]:
start_time = time.time()

for idx in range(100): #check execution time of finding vector
    get_title_vector('Mathematicia')
    
time.time()-start_time

26.1965069770813

## Get job vectors for 13,000+ reviews

We will now find the vector representations of over 13,000 jobs. To guard against job titles that have been misspelled or our erroneous, we will only consider job titles that have been reviewed at least three at some company. (So I would include a job title if it was reviewed three times at one company and no times at every other company.)

We will store this in a DataFrame with the rows being job titles and columns being ratings of skills required for that job.

In [ ]:
#DataFrame of all positions reviewed at companies, with number of reviews
company_job_counts = pd.read_csv('companies_jobs_counts.csv',
                                 index_col='Unnamed: 0')

In [20]:
#only consider jobs with at least 3 reviews at some company
min_reviews_at_a_company = 3
company_job_counts = company_job_counts[company_job_counts['count'] >= min_reviews_at_a_company]

print(company_job_counts.shape)

company_job_counts.head()

(92216, 5)


,Company Id,company_name,Job Title,Employee Status,count
5,4,AAR,A&P Mechanic,Former Employee,5
6,4,AAR,A&P Technician,Current Employee,3
15,4,AAR,Aircraft Mechanic,Current Employee,3
16,4,AAR,Aircraft Mechanic,Former Employee,3
22,4,AAR,Anonymous Contractor,Former Contractor,6


In [21]:
#look at number of distinct jobs
total_number_of_titles = company_job_counts.loc[:,'Job Title'].nunique() #183667

total_number_of_titles

13379

In [22]:
jobs_at_least_n_reviews = company_job_counts.loc[:,'Job Title'].unique().tolist()

In [23]:
#remove 'anonymous' job titles
for titl in ['Anonymous Employee', 'Anonymous Intern', 'Anonymous Contractor']:
    jobs_at_least_n_reviews.remove(titl)

In [26]:
start_time = time.time()
counter = 0

jobs_series = []
job_names = []

for job in jobs_at_least_n_reviews:
    try:
        job_names.append(job)
        jobs_series.append(get_title_vector(job))
        counter += 1
    except IndexError as error:#jobs that lead to error when searched
        print(counter)
        print(str(time.time() - start_time) + ' seconds.')
        print(job)
        print('-'*50)

139
51.685861110687256 seconds.
Anonymous
--------------------------------------------------
252
102.89835524559021 seconds.
Karnataka
--------------------------------------------------
574
220.05006790161133 seconds.
HRBP
--------------------------------------------------
640
245.0918321609497 seconds.
BDM
--------------------------------------------------
687
262.7247312068939 seconds.
Texas
--------------------------------------------------
765
292.34693717956543 seconds.
CSSR
--------------------------------------------------
778
297.42057704925537 seconds.
CSSC
--------------------------------------------------
983
381.26595091819763 seconds.
MCSA
--------------------------------------------------
1013
391.3919689655304 seconds.
nan
--------------------------------------------------
1076
415.3689351081848 seconds.
Csr1
--------------------------------------------------
1132
436.74820613861084 seconds.
MP&P Engineer
--------------------------------------------------
1583
609.286451

6858
2671.4487810134888 seconds.
Sdl
--------------------------------------------------
6929
2697.566085100174 seconds.
Barita
--------------------------------------------------
6929
2697.760584115982 seconds.
Batista
--------------------------------------------------
7075
2756.1417059898376 seconds.
LSP
--------------------------------------------------
7295
2860.1716890335083 seconds.
A2L
--------------------------------------------------
7361
2886.4959061145782 seconds.
Bezorger
--------------------------------------------------
7437
2916.613368988037 seconds.
Fpa
--------------------------------------------------
7454
2924.192598104477 seconds.
GRS
--------------------------------------------------
7544
2955.5382339954376 seconds.
Paketzusteller
--------------------------------------------------
7590
2971.100677013397 seconds.
SBL
--------------------------------------------------
7595
2973.031126022339 seconds.
Ingénieur Stagiaire
--------------------------------------------------

11129
4360.001549005508 seconds.
BAM
--------------------------------------------------
11133
4361.417860031128 seconds.
SBAE
--------------------------------------------------
11492
4486.744343996048 seconds.
ACSSM
--------------------------------------------------
11551
4507.418752908707 seconds.
SSHA
--------------------------------------------------
11608
4529.096888065338 seconds.
SMM
--------------------------------------------------
11635
4540.347991943359 seconds.
HCA
--------------------------------------------------
11739
4577.869448900223 seconds.
DST
--------------------------------------------------
11902
4638.465209007263 seconds.
USO
--------------------------------------------------
11902
4638.689711093903 seconds.
USO G4S
--------------------------------------------------
11922
4647.013431072235 seconds.
DTL
--------------------------------------------------
11997
4676.0709500312805 seconds.
Filialleiter
--------------------------------------------------
12035
4690.634

In [27]:
all_jobs_ratings = pd.concat([jobs_series[idx].rename(job_names[idx]) for idx in range(len(jobs_series))], axis=1)

#all_jobs_ratings = pd.concat([get_title_vector(job).rename(job) for job in jobs_at_least_n_reviews], axis=1)

In [29]:
all_jobs_ratings.to_csv('all_jobs_ratings.csv')

In [33]:
all_jobs_ratings.head()

,A&P Mechanic,A&P Technician,Aircraft Mechanic,Avionics Technician,Engineer,Sheet Metal Mechanic,Senior Software Engineer,Agent,Aircraft Maintenance Supervisor,Aircraft Maintenance Technician,Airline Pilot,Airport Agent,American Airlines Flight Attendant,Amt,Analyst,Aviation Maintenance Technician,Baggage Handler,Business Analyst,CSA,CSM,Captain,"Captain, Major Airline",Commodity Manager,Crew Chief,Crew Scheduler,Customer Assistance Representative,Customer Relations Representative,Customer Service,Customer Service Agent,Customer Service Manager,Customer Service Representative,Financial Analyst,First Officer,Fleet Service,Fleet Service Agent,Fleet Service Clerk,Flight Attendant,Gate Agent,Home Based Sales Representative,Human Resources,International Flight Attendant,Manager,Mechanic,Passenger Service Agent,Passenger Service Representative,Pilot,"Pilot, First Officer",Pilot/Captain,Project Manager,Ramp Agent,Res Agent,Reservation Representative,Reservationist,Reservations Agent,Senior Analyst,Senior Commodity Manager,Senior Financial Analyst,Senior Manager,Senior Programmer Analyst,Senior Project Manager,Software Developer,Staff Assistant,Station Agent,Strategy Analyst,Ticket Agent,AREA SALES MANAGER,Associate Scientist,Business Systems Analyst,Director,District Sales Manager,Engineering Intern,Field Service Engineer,Finance Manager,IT Manager,Intern,Marketing Manager,Pharmaceuticals Sales Representative,Product Manager,Program Manager,Quality,Quality Engineer,Sales,Sales Representative,Sales Specialist,Scientist,Senior Business Systems Analyst,Senior Engineer,Senior IT Project Manager,Senior Research Scientist,Senior Scientist,Senior Systems Analyst,Systems Analyst,Technical Specialist,Territory Manager,Head Teller,Relationship Banker,ASIC Design Engineer,ASIC/Layout Design Engineer II,Associate Engineer,Co-Op,Co-Op Engineer,Design Engineer,Design Engineer I,Design Engineer II,Director of Engineering,Engineering Manager,Financial Accountant,Member of Technical Staff,Member of Technical Staff Design Engineer,Member of Technical Staff Software Engineer,PEY,PEY Engineer,PMTS,Principal Design Engineer,Principal Member of Technical Staff,Product Development Engineer,Product Development Engineer II,Product Marketing Manager,SMTS Design Engineer,SMTS Software Engineer,Senior ASIC Design Engineer,Senior ASIC/Layout Design Engineer,Senior Design Engineer,Senior Hardware Engineer,Senior Member of Technical Staff,Senior Product Development Engineer,Senior Product Marketing Manager,Senior Program Manager,Senior Software Development Engineer,Smts,Software,Software Development Engineer II,Software Engineer,Software Engineer II,Verification Engineer,Account Executive,Account Manager,Actuarial Consultant,Administrative Assistant,Anonymous,Business Project Program Manager,CMA,Care Management Associate,Care Manager,Case Manager,Case Manager (RN),Claim Benefit Specialist,Claims,Claims Analyst,Claims Processor,Claims Specialist,Clinical Care Manager,Clinical Case Manager,Consultant,Data Scientist,Finance Director,Financial Manager,Health Concierge,IT Project Manager,Learning Consultant,Management,Medicare Customer Service Representative,Member Service Representative,Member Services,Network Account Manager,Nurse Case Manager,Nurse Consultant,Operations Manager,Project Lead,Provider Service Specialist,RN Case Manager,Registered Nurse,Sales Consultant,Senior Applications Developer,Senior Business Analyst,Senior Consultant,Senior Customer Service Representative,Senior Director,Senior Software Developer,Senior Underwriting Analyst,Summer Associate,Supervisor,Systems Engineer,UM Nurse Consultant,Underwriter,Underwriting Analyst,Underwriting Consultant,Utilization Management,Utilization Review Nurse,Wellness Coach,Class A Truck Driver,Microbulk Sales Representative,Operator,Plant Operator,Project Engineer,Branch Manager,Bulk Driver,Cash Applications Specialist,Delivery Driver,Driver,Inside Sales,Inside Sales Representative,Plant Manager,Production Operat

In [43]:
#rows: job titles
#columns: skills required for job
all_jobs_ratings.transpose().to_csv('all_jobs_ratings_transpose.csv')

In [51]:
#number of companies represented
company_job_counts.loc[:,'Company Id'].nunique()

5832